In [1]:
%%time
%matplotlib inline
import importlib
import new_import  
importlib.reload(new_import)
from new_import import *

CPU times: user 8.26 s, sys: 5.98 s, total: 14.2 s
Wall time: 8.17 s


In [2]:
cluster, client = notebook_utils.initialize_dask(use_gateway=True, workers=(1,10))
# Khai báo 1 Datacube là dc
dc = datacube.Datacube()
client

Starting new cluster.


Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: https://hub.asia.easi-eo.solutions/services/dask-gateway/clusters/easihub.e3d2ac5f92b344919f7ff2018837c8f4/status,


In [3]:
date_range = ('2022-09-01', '2023-10-01')
longtitude_range = (105.5, 106.4)
latitude_range = (9.2, 10.0) 

coordinates = (longtitude_range, latitude_range)

In [4]:
dsvh, dsvv = load_data_sen1(dc, date_range, coordinates)
data_sen2 = load_data_sen2(dc, date_range, coordinates)
notebook_utils.heading(notebook_utils.xarray_object_size(data_sen2))
display(data_sen2)

<xarray.Dataset>
Dimensions:      (time: 33, y: 8874, x: 9902)
Coordinates:
  * time         (time) datetime64[ns] 2022-09-06T22:46:14.500000 ... 2023-09...
  * y            (y) float64 1.106e+06 1.106e+06 ... 1.017e+06 1.017e+06
  * x            (x) float64 5.548e+05 5.548e+05 ... 6.538e+05 6.538e+05
    spatial_ref  int32 32648
Data variables:
    vv           (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    vh           (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
Attributes:
    crs:           EPSG:32648
    grid_mapping:  spatial_ref

No datasets require offset correction
The valid_data_mask and scale (no offset) have been applied to the reflectance bands


<xarray.Dataset>
Dimensions:      (time: 151, y: 8874, x: 9902)
Coordinates:
  * time         (time) datetime64[ns] 2022-09-02T03:35:23.960000 ... 2023-09...
  * y            (y) float64 1.106e+06 1.106e+06 ... 1.017e+06 1.017e+06
  * x            (x) float64 5.548e+05 5.548e+05 ... 6.538e+05 6.538e+05
    spatial_ref  int32 32648
Data variables:
    red          (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    nir          (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    scl          (time, y, x) uint8 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
Attributes:
    crs:           EPSG:32648
    grid_mapping:  spatial_ref

In [5]:
average_vv = calculate_average(dsvv, time_pattern='1M')
progress(average_vv)

VBox()

In [6]:
average_vh = calculate_average(dsvh, time_pattern='1M')
progress(average_vh)

VBox()

In [7]:
# Tiến hành loại bỏ các vị trí bị mây ảnh hưởng
result = mask_cloud(data_sen2)
progress(result)

{'0': 'no data',
 '1': 'saturated or defective',
 '2': 'dark area pixels',
 '3': 'cloud shadows',
 '4': 'vegetation',
 '5': 'bare soils',
 '6': 'water',
 '7': 'unclassified',
 '8': 'cloud medium probability',
 '9': 'cloud high probability',
 '10': 'thin cirrus',
 '11': 'snow or ice'}

VBox()

In [8]:
# calculate NDVI
dsNDVI = calculate_indices(result, index='NDVI', satellite_mission='s2')
ndvi = dsNDVI["NDVI"]

In [9]:
# calculate average NDVI
average_ndvi = calculate_average(ndvi,time_pattern="1M")
progress(average_ndvi)

VBox()

In [10]:
%%time
average_vh = average_vh.compute()
average_vv = average_vv.compute()
average_ndvi = average_ndvi.compute()


CPU times: user 13.7 s, sys: 9 s, total: 22.7 s
Wall time: 13min 50s


In [ ]:
plt.imshow(average_ndvi.isel(time=6), vmin=None, vmax=None)
average_ndvi.shape

SAVE THE DATA FOR TRAINING SERVERAL TIME

In [31]:
np.savez('input/saved_datasets/full_dataset_cloud.npz', ndvi=average_ndvi, vh=average_vh, vv=average_vv)

LOAD SAVED DATASET TO USE

In [11]:
full_dataset = np.load('input/saved_datasets/full_dataset_cloud.npz')

In [12]:
ndvi = full_dataset['ndvi']
vh = full_dataset['vh']
vv = full_dataset['vv']

In [13]:
mask = ~np.isnan(ndvi)
X_train = np.stack([vh[mask], vv[mask]], axis=1)
y_train = ndvi[mask]

In [14]:
# np.savez('datasets/train_dataset/full_data_regressor_train.npz', data=X_train, label=y_train)

In [15]:
num_samples = 2000000

# Generate random indices
random_indices = np.random.choice(X_train.shape[0], size=num_samples, replace=False)

# Subset the data using the random indices
X_train_subset = X_train[random_indices]
y_train_subset = y_train[random_indices]

# Check the new shape
print(X_train_subset.shape)
print(y_train_subset.shape)


(2000000, 2)
(2000000,)


In [16]:
# np.savez('input/saved_datasets/2m_cloud_dataset.npz', data=X_train_subset, label=y_train_subset)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_train_subset, y_train_subset, test_size=0.2, random_state=42)

TRAIN MODEL

In [20]:
%%time
best_model, best_params, best_score_r2, best_score_mae, cv_results = find_best_regression(
    X_train, y_train, regressors['knn'], param_grids_regressor['knn']
)

Best Parameters: {'n_neighbors': 50, 'p': 1, 'weights': 'uniform'}
Best R² Score: 0.7388694275303209
Best MAE Score: -0.17420636117458344
CPU times: user 1.98 s, sys: 1.46 s, total: 3.44 s
Wall time: 3min 56s


In [21]:
folder_path = 'output/regressors/'
file_path = os.path.join(folder_path, 'knn_cloud.pkl')
joblib.dump(best_model, file_path)

['output/regressors/knn_cloud.pkl']

In [ ]:
y_pred = best_model.predict(X_test)
# Calculate R² score
r2 = r2_score(y_test, y_pred)
# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f"R² Score: {r2}")
print(f"Mean Squared Error: {mse}")

print("Mean Absolute Error:", mae)

FILL CLOUD 

In [ ]:
best_model = joblib.load("output/regressors/rf_cloud.pkl")

In [ ]:
filled_ndvi = np.copy(ndvi)

In [ ]:
X_pred = np.stack([vh[~mask], vv[~mask]], axis=1)
filled_ndvi[~mask] = best_model.predict(X_pred)

In [ ]:
filled_ndvi[~mask]

In [ ]:
np.savez('input/saved_datasets/rf_full_dataset_filled.npz', ndvi=filled_ndvi, vv=vv, vh=vh)

In [ ]:
client.close()
cluster.close()